In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.models as models
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import math
import csv
import os

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

torch.manual_seed(0)

def get_transform():
    normalize = [(0.12, 0.12, 0.12), (0.19, 0.19, 0.19)]
    data_transforms = transforms.Compose(
        [
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(*normalize),
        ]
    )
    return data_transforms

print("Load resnet")
# not pretrained
model = models.resnet18(pretrained=False)
num_classes = 4  # Number of categories
model.fc = nn.Linear(model.fc.in_features, num_classes)

criterion = nn.CrossEntropyLoss()
# optimizer = optim.Adam(model.parameters(), lr=0.001) # not as good
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Specify the dataset directory
data_dir = '/mnt/c/Users/Sean/Downloads/OCT_DATASET_DO_NOT_SHARE_WITH_ANYONE_split/'
train_dataset = datasets.ImageFolder(data_dir + 'train', transform=get_transform())
test_dataset = datasets.ImageFolder(data_dir + 'test', transform=get_transform())
val_dataset = datasets.ImageFolder(data_dir + 'val', transform=get_transform())

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

model.to(device)
train_loss_arr, train_acc_arr, val_acc_arr, test_acc_arr = [], [], [], []

print("Start training")
name = "resnet18_batch64_noaug_nopretrain"

num_epochs = 50
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f'Epoch {epoch + 1}, Loss: {running_loss / len(train_loader)}', end='  |  ')
    train_loss_arr.append(running_loss / len(train_loader))

    model.eval()

    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f'Train Accuracy: {accuracy:.2f}%', end='  |  ')
    train_acc_arr.append(accuracy)

    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f'Test Accuracy: {accuracy:.2f}%', end='  |  ')
    test_acc_arr.append(accuracy)

    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f'Val Accuracy: {accuracy:.2f}%')
    val_acc_arr.append(accuracy)

torch.save(model.state_dict(), 'resnet18_e20.ckpt')



root_path = "./csv"
if not os.path.exists(root_path):
    os.makedirs(root_path)
    
with open(f"{root_path}/{name}.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerow(["train_loss", "train_acc", "test_acc", "val_acc"])
    for i in range(len(train_loss_arr)):
        writer.writerow([train_loss_arr[i], train_acc_arr[i], test_acc_arr[i], val_acc_arr[i]])

Load resnet
Start training
Epoch 1, Loss: 1.4083633224169414  |  Train Accuracy: 35.77%  |  Test Accuracy: 34.69%  |  Val Accuracy: 35.71%
Epoch 2, Loss: 1.374448537826538  |  Train Accuracy: 35.77%  |  Test Accuracy: 34.69%  |  Val Accuracy: 35.71%
Epoch 3, Loss: 1.359290103117625  |  Train Accuracy: 35.77%  |  Test Accuracy: 34.69%  |  Val Accuracy: 35.71%
Epoch 4, Loss: 1.3447126150131226  |  Train Accuracy: 35.49%  |  Test Accuracy: 36.73%  |  Val Accuracy: 33.33%
Epoch 5, Loss: 1.3236403067906697  |  Train Accuracy: 35.77%  |  Test Accuracy: 34.69%  |  Val Accuracy: 35.71%
Epoch 6, Loss: 1.2980592846870422  |  Train Accuracy: 36.62%  |  Test Accuracy: 34.69%  |  Val Accuracy: 38.10%
Epoch 7, Loss: 1.2714688579241435  |  Train Accuracy: 40.28%  |  Test Accuracy: 38.78%  |  Val Accuracy: 38.10%
Epoch 8, Loss: 1.2500166098276775  |  Train Accuracy: 38.31%  |  Test Accuracy: 34.69%  |  Val Accuracy: 38.10%
Epoch 9, Loss: 1.221095581849416  |  Train Accuracy: 38.59%  |  Test Accuracy: 